In [66]:
import sys
import json
import base64
import numpy as np
from scipy import misc

In [67]:
from pathlib import Path

In [68]:
PATH = Path.cwd()

In [69]:
Path.home()

PosixPath('/home/paperspace')

In [70]:
# student_output = sys.argv[1]
student_output = PATH/'tester_data_crop'

In [71]:
import torch

In [51]:
def decode(packet):
	img = base64.b64decode(packet)
	filename = PATH/'image.png'
	with open(filename, 'wb') as f:
			f.write(img)
	result = misc.imread(filename)
	return result

with open(PATH/'results.json') as json_data:
	ans_data = json.loads(json_data.read())
	json_data.close()

# Load student data
with open(student_output) as student_data:
	student_ans_data = json.loads(student_data.read())
	student_data.close()

frames_processed = 0

Car_TP = 0 # True Positives
Car_FP = 0 # Flase Positives
Car_TN = 0 # True Negatives
Car_FN = 0 # True Negatives

Road_TP = 0 # True Positives
Road_FP = 0 # Flase Positives
Road_TN = 0 # True Negatives
Road_FN = 0 # True Negatives

tc = []
tr = []
pc = []
pr = []
for frame in range(1,len(ans_data.keys())+1):

    truth_data_car =  decode(ans_data[str(frame)][0])
    truth_data_road =  decode(ans_data[str(frame)][1])
    student_data_car = decode(student_ans_data[str(frame)][0])
    student_data_road = decode(student_ans_data[str(frame)][1])
    tc.append(truth_data_car)
    tr.append(truth_data_road)
    pc.append(student_data_car)
    pr.append(student_data_road)

/home/paperspace/anaconda3/envs/lyft/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [52]:
def f2_score(y_true, y_pred, threshold=0.5):
    return fbeta_score(y_true, y_pred, 2, threshold)


def fbeta_score(y_true, y_pred, beta, threshold=None, eps=1e-9):
    beta2 = beta**2

    if threshold:
        y_pred = torch.ge(y_pred.float(), threshold).float()
    else:
        y_pred = y_pred.float()
    y_true = y_true.float()

    true_positive = (y_pred * y_true).sum()
    precision = true_positive/(y_pred.sum()+(eps))
    recall = true_positive/(y_true.sum()+eps)
    
    fb = (precision*recall)/(precision*beta2 + recall + eps)*(1+beta2)
    
    return fb, true_positive, precision, recall

In [53]:
tc_s = np.stack(tc)
tr_s = np.stack(tr)
pc_s = np.stack(pc)
pr_s = np.stack(pr)

In [54]:
y_true = torch.from_numpy(tc_s).float()
y_pred = torch.from_numpy(pc_s).float()

In [55]:
# true_positive = (y_pred * y_true).sum()
eps=1e-9
true_positive = (y_pred * y_true).sum()
precision = true_positive/(y_pred.sum()+(eps))
recall = true_positive/(y_true.sum()+eps)

In [56]:
beta2 = 2

In [57]:
fb = (precision*recall)/(precision*beta2 + recall + eps)*(1+beta2)
#     div_(precision.mul(beta2) + recall + eps).
#     mul_(1 + beta2))

In [58]:
fb

0.544933755922913

In [59]:
precision

0.369281934231573

In [60]:
recall

0.7149756088603755

In [61]:
true_positive

116812.0

In [62]:
fbeta_score(torch.from_numpy(tr_s), torch.from_numpy(pr_s), .5)

(0.9869345795946747, 2912591.0, 0.9873300277596323, 0.9853559535799356)

In [63]:
fbeta_score(torch.from_numpy(tc_s), torch.from_numpy(pc_s), 2)

(0.6022242887302403, 116812.0, 0.369281934231573, 0.7149756088603755)

In [64]:
Car_TP, Car_FP, Car_TN, Car_FN

(0, 0, 0, 0)

In [72]:
def decode(packet):
	img = base64.b64decode(packet)
	filename = PATH/'image.png'
	with open(filename, 'wb') as f:
			f.write(img)
	result = misc.imread(filename)
	return result

with open(PATH/'results.json') as json_data:
	ans_data = json.loads(json_data.read())
	json_data.close()

# Load student data
with open(student_output) as student_data:
	student_ans_data = json.loads(student_data.read())
	student_data.close()

frames_processed = 0

Car_TP = 0 # True Positives
Car_FP = 0 # Flase Positives
Car_TN = 0 # True Negatives
Car_FN = 0 # True Negatives

Road_TP = 0 # True Positives
Road_FP = 0 # Flase Positives
Road_TN = 0 # True Negatives
Road_FN = 0 # True Negatives

for frame in range(1,len(ans_data.keys())+1):

	truth_data_car =  decode(ans_data[str(frame)][0])
	truth_data_road =  decode(ans_data[str(frame)][1])
	student_data_car = decode(student_ans_data[str(frame)][0])
	student_data_road = decode(student_ans_data[str(frame)][1])

	Car_TP += np.sum(np.logical_and(student_data_car == 1, truth_data_car == 1))
	Car_FP += np.sum(np.logical_and(student_data_car == 1, truth_data_car == 0))
	Car_TN += np.sum(np.logical_and(student_data_car == 0, truth_data_car == 0))
	Car_FN += np.sum(np.logical_and(student_data_car == 0, truth_data_car == 1))

	Road_TP += np.sum(np.logical_and(student_data_road == 1, truth_data_road == 1))
	Road_FP += np.sum(np.logical_and(student_data_road == 1, truth_data_road == 0))
	Road_TN += np.sum(np.logical_and(student_data_road == 0, truth_data_road == 0))
	Road_FN += np.sum(np.logical_and(student_data_road == 0, truth_data_road == 1))

	frames_processed+=1


# Generate results
Car_precision = Car_TP/(Car_TP+Car_FP)/1.0
Car_recall = Car_TP/(Car_TP+Car_FN)/1.0
Car_beta = 2
Car_F = (1+Car_beta**2) * ((Car_precision*Car_recall)/(Car_beta**2 * Car_precision + Car_recall))
Road_precision = Road_TP/(Road_TP+Road_FP)/1.0
Road_recall = Road_TP/(Road_TP+Road_FN)/1.0
Road_beta = 0.5
Road_F = (1+Road_beta**2) * ((Road_precision*Road_recall)/(Road_beta**2 * Road_precision + Road_recall))

print ("Car F score: %05.3f  | Car Precision: %05.3f  | Car Recall: %05.3f  |\n\
Road F score: %05.3f | Road Precision: %05.3f | Road Recall: %05.3f | \n\
Averaged F score: %05.3f" %(Car_F,Car_precision,Car_recall,Road_F,Road_precision,Road_recall,((Car_F+Road_F)/2.0)))

/home/paperspace/anaconda3/envs/lyft/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


Car F score: 0.721  | Car Precision: 0.445  | Car Recall: 0.853  |
Road F score: 0.996 | Road Precision: 0.996 | Road Recall: 0.994 | 
Averaged F score: 0.858


In [ ]:
# With new data and data loader
Car F score: 0.560  | Car Precision: 0.476  | Car Recall: 0.586  |
Road F score: 0.982 | Road Precision: 0.982 | Road Recall: 0.981 | 
Averaged F score: 0.771

In [ ]:
# Best updated
Car F score: 0.685  | Car Precision: 0.709  | Car Recall: 0.679  |
Road F score: 0.987 | Road Precision: 0.987 | Road Recall: 0.987 | 
Averaged F score: 0.836

In [ ]:
# Padding with no crop
Car F score: 0.657  | Car Precision: 0.718  | Car Recall: 0.643  |
Road F score: 0.987 | Road Precision: 0.987 | Road Recall: 0.987 | 
Averaged F score: 0.822

In [ ]:
# Real model
# 226:54
Car F score: 0.613  | Car Precision: 0.721  | Car Recall: 0.591  |
Road F score: 0.988 | Road Precision: 0.990 | Road Recall: 0.981 | 
Averaged F score: 0.801

In [ ]:
# Real model
# 194:54
Car F score: 0.600  | Car Precision: 0.599  | Car Recall: 0.600  |
Road F score: 0.982 | Road Precision: 0.981 | Road Recall: 0.985 | 
Averaged F score: 0.791

In [ ]:
# 24x64
Car F score: 0.574  | Car Precision: 0.722  | Car Recall: 0.546  |
Road F score: 0.983 | Road Precision: 0.992 | Road Recall: 0.946 | 
Averaged F score: 0.778

In [ ]:
# 24x0
Car F score: 0.609  | Car Precision: 0.692  | Car Recall: 0.591  |
Road F score: 0.989 | Road Precision: 0.990 | Road Recall: 0.983 | 
Averaged F score: 0.799

In [ ]:
# 200x80
Car F score: 0.477  | Car Precision: 0.769  | Car Recall: 0.435  |
Road F score: 0.919 | Road Precision: 0.994 | Road Recall: 0.705 | 
Averaged F score: 0.698